In [4]:
from scipy.sparse import eye
from scipy.sparse.linalg import expm_multiply
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import norm

import numpy as np
from resource_estimate_utils import *
from os.path import join
from time import time

from qiskit.quantum_info import SparsePauliOp
from qiskit.synthesis import LieTrotter, SuzukiTrotter
from qiskit import transpile
from qiskit.circuit.library import PauliEvolutionGate
from pytket import OpType
from pytket.passes import RemoveRedundancies, CommuteThroughMultis, SequencePass, FullPeepholeOptimise, auto_rebase_pass
from pytket.extensions.qiskit import qiskit_to_tk

from braket.devices import LocalSimulator
import networkx as nx
from random import shuffle, seed
from scipy.sparse import diags

from braket.devices import LocalSimulator

import sys
from os.path import join
sys.path.append(join("..", ".."))
from utils import *

# QW on glued trees

In [5]:
def get_glued_tree(h):
    seed(0)
    # Two binary trees of height h (2^(h+1) - 1 nodes each glued together)
    num_nodes_per_binary_tree = 2 ** (h+1) - 1
    num_nodes = 2 * num_nodes_per_binary_tree
    graph = nx.Graph()

    # Leaves
    leaves_first = []
    leaves_second = []
    for i in range(2 ** h):
        leaves_first.append(2 ** h - 1 + i)
        leaves_second.append(num_nodes - 1 - (2 ** h - 1) - i)

    for i in np.arange(1, num_nodes_per_binary_tree):

        # First binary tree
        graph.add_edge(int((i-1)/2), i)

        # Second binary tree
        graph.add_edge(num_nodes - 1 - int((i-1)/2), num_nodes - 1 - i)

    # Glue the two trees together
    # Shuffle the leaves to get a random cycle
    shuffle(leaves_first)
    shuffle(leaves_second)

    for i in range(2 ** h):
        graph.add_edge(leaves_first[i], leaves_second[i])
        graph.add_edge(leaves_second[i], leaves_first[(i+1) % (2 ** h)])

    return graph

def get_H_pauli_op(n, graph):

    H = []

    for i,j in graph.edges():
        op = n * ['I']
        op[i] = 'X'
        op[j] = 'X'
        H.append(SparsePauliOp(''.join(op), 1/2))

        op = n * ['I']
        op[i] = 'Y'
        op[j] = 'Y'
        H.append(SparsePauliOp(''.join(op), 1/2))

    return sum(H).simplify()

def get_binary_resource_estimate(h, s, error_tol, trotter_method, num_samples, num_jobs):
    
    T = get_T_s(h, s)
    print(f"T={T}")
    graph = get_glued_tree(h)
    N = graph.order()
    print(f"h = {h}, N = {N}", flush=True)

    adjacency_matrix = nx.adjacency_matrix(graph, nodelist=sorted(graph.nodes())).toarray()
    adjacency_matrix_padded = np.pad(adjacency_matrix, (0, 2 ** int(np.ceil(np.log2(N))) - N))

    pauli_op = SparsePauliOp.from_operator(adjacency_matrix_padded)

    # Compute number of gates per Trotter step
    if trotter_method == "first_order" or trotter_method == "randomized_first_order":
        circuit = LieTrotter(reps=1).synthesize(PauliEvolutionGate(pauli_op.group_commuting()))
    elif trotter_method == "second_order":
        circuit = SuzukiTrotter(order=2, reps=1).synthesize(PauliEvolutionGate(pauli_op.group_commuting()))
    else:
        raise ValueError(f"{trotter_method} not supported")
    
    compiled_circuit = transpile(circuit, basis_gates=['rxx', 'rx', 'ry', 'rz'], optimization_level=3)
    tket_circuit = qiskit_to_tk(compiled_circuit)
    gateset = {OpType.Rx, OpType.Ry, OpType.Rz, OpType.XXPhase}
    rebase = auto_rebase_pass(gateset) 
    comp = SequencePass([FullPeepholeOptimise(), CommuteThroughMultis(), RemoveRedundancies(), rebase])
    comp.apply(tket_circuit)

    # Gates per Trotter step
    num_single_qubit_gates, num_two_qubit_gates = tket_circuit.n_1qb_gates(), tket_circuit.n_2qb_gates()
    print(f"1q gates: {num_single_qubit_gates}, 2q gates: {num_two_qubit_gates}")

    # Estimate number of Trotter steps required
    r_min, r_max = 1, 10
    while std_bin_trotter_error_sampling(adjacency_matrix_padded, pauli_op, T, r_max, trotter_method, num_samples, num_jobs) > error_tol:
        r_max *= 2

    # binary search for r
    while r_max - r_min > 1:
        r = (r_min + r_max) // 2
        if std_bin_trotter_error_sampling(adjacency_matrix_padded, pauli_op, T, r, trotter_method, num_samples, num_jobs) > error_tol:
            r_min = r
        else:
            r_max = r
    
    print(f"Finished N={N}, num two qubit gates={num_two_qubit_gates}, trotter steps={r_max}", flush=True)
    return num_single_qubit_gates, num_two_qubit_gates, r_max

def get_weighted_chain_hamiltonian(h):
    # weighted chain with 2 * (h + 1) vertices
    num_vertices = 2 * (h + 1)
    weights = np.sqrt(2) * np.ones(num_vertices - 1)
    weights[int((num_vertices - 1) / 2)] = 2
    return diags([weights, weights], offsets=[1,-1])

def get_T_s(h, s):
    num_time_points = 256
    T = h + 1
    t_vals = np.linspace(0, T, num_time_points)

    H_weighted_chain = get_weighted_chain_hamiltonian(h)
    psi_0 = np.zeros(2 * (h + 1))
    psi_0[0] = 1
    psi = expm_multiply(-1j * H_weighted_chain, psi_0, start=0, stop=T, num=num_time_points)
    dist = np.abs(psi) ** 2
    return t_vals[np.argmax(dist[:,-1] > s)]



In [6]:
print("Resource estimation for QW on glued tree.")

num_jobs = 64
print("Number of jobs:", num_jobs)
num_samples = 5000

dimension = 1
trotter_method = "randomized_first_order"

s = 0.4
print(f"Probability threshold s = {s : 0.2f}")
# print(f"Error tolerance: {error_tol : 0.2f}.")
print(f"Method: {trotter_method}")

Resource estimation for QW on glued tree.
Number of jobs: 64
Probability threshold s =  0.40
Method: randomized_first_order


In [7]:
h = 2
encoding = "one-hot"
device = LocalSimulator()
r = 4

T = get_T_s(h, s)
print(f"T={T}")
graph = get_glued_tree(h)
N = graph.order()
print(f"N={N}")
n = num_qubits_per_dim(N, encoding)
codewords = get_codewords(N, dimension, encoding, periodic=False)

print(f"Running N = {N} for {encoding}", flush=True)
adjacency_matrix = nx.adjacency_matrix(graph, nodelist=sorted(graph.nodes())).toarray()

H_terms, graph = get_H_terms_one_hot(N, adjacency_matrix)
H_ebd = sum(H_terms)

# Estimate number of Trotter steps required
error_tol = estimate_trotter_error(N, H_ebd, T, get_one_hot_circuit(N, adjacency_matrix, T, r, trotter_method), dimension, encoding, codewords, device, num_samples, num_jobs)

T=2.0
N=14
Running N = 14 for one-hot


In [8]:
print("\nRunning resource estimation for standard binary encoding")
h = 2

binary_one_qubit_gate_count_per_trotter_step, binary_two_qubit_gate_count_per_trotter_step, binary_trotter_steps = get_binary_resource_estimate(h, s, error_tol, trotter_method, num_samples, num_jobs)
    
print("Std binary gate counts")
print(binary_one_qubit_gate_count_per_trotter_step * binary_trotter_steps, binary_two_qubit_gate_count_per_trotter_step * binary_trotter_steps)



Running resource estimation for standard binary encoding
T=2.0
h = 2, N = 14
1q gates: 1522, 2q gates: 233
Finished N=14, num two qubit gates=233, trotter steps=4
Std binary gate counts
6088 932


# Spatial search

In [9]:
def get_T_2d(N, H_spatial_search):
    dimension = 2
    T = N ** dimension
    num_time_points = 256
    t_vals = np.linspace(0, T, num_time_points)
    p = 4 * (np.log(N) / N) ** 2
    psi_0 = np.ones(N ** dimension)
    psi_0 /= np.linalg.norm(psi_0)

    psi = expm_multiply(-1j * H_spatial_search, psi_0, start=0, stop=T, num=num_time_points)
    dist = np.abs(psi) ** 2

    return t_vals[np.argmax(dist[:,-N] >= p)]

def get_binary_resource_estimate(N, dimension, error_tol, trotter_method, num_samples, num_jobs):
    print(f"N = {N}", flush=True)

    # Compute the optimal gamma
    L = get_laplacian_lattice(N, d=dimension)
    marked_vertex_1 = np.zeros(N)
    marked_vertex_1[N-1] = 1
    marked_vertex_2 = np.zeros(N)
    marked_vertex_2[0] = 1

    marked_vertex = np.kron(marked_vertex_1, marked_vertex_2)
    H_oracle = - csc_matrix(np.outer(marked_vertex, marked_vertex))

    # Sign is flipped here; this function minimizes the difference between the two largest eigenvalues of gamma * L + H_oracle
    gamma = scipy_get_optimal_gamma(L, -H_oracle, 0.3)
    H_spatial_search = (-gamma * L + H_oracle).toarray()

    T = get_T_2d(N, H_spatial_search)

    N_padded = 2 ** int(np.ceil(np.log2(N)))

    L = lil_matrix(get_laplacian_lattice(N_padded, d=dimension))
    binary_indices = []
    other_indices = []
    for i in range(N_padded ** dimension):
        col = i % N_padded
        row = i // N_padded
        if col < N and row < N:
            binary_indices.append(i)
        else:
            other_indices.append(i)

    H_spatial_search_padded = -gamma * L
    # Make sure the diagonal part is the same
    for i in range(len(binary_indices)):
        for j in range(len(binary_indices)):
            H_spatial_search_padded[binary_indices[i], binary_indices[j]] = H_spatial_search[i,j]
        
    # Remove all other edges
    for i in binary_indices:
        for j in other_indices:
            H_spatial_search_padded[i,j] = 0
            H_spatial_search_padded[j,i] = 0
    # for i in other_indices:
    #     for j in other_indices:
    #         H_spatial_search_padded[i,j] = 0

    pauli_op = SparsePauliOp.from_operator(H_spatial_search_padded.toarray())
    
    # Compute number of gates per Trotter step
    if trotter_method == "first_order" or trotter_method == "randomized_first_order":
        circuit = LieTrotter(reps=1).synthesize(PauliEvolutionGate(pauli_op.group_commuting()))
    elif trotter_method == "second_order":
        circuit = SuzukiTrotter(order=2, reps=1).synthesize(PauliEvolutionGate(pauli_op.group_commuting()))
    else:
        raise ValueError(f"{trotter_method} not supported")

    compiled_circuit = transpile(circuit, basis_gates=['rxx', 'rx', 'ry', 'rz'], optimization_level=3)
    tket_circuit = qiskit_to_tk(compiled_circuit)
    gateset = {OpType.Rx, OpType.Ry, OpType.Rz, OpType.XXPhase}
    rebase = auto_rebase_pass(gateset) 
    comp = SequencePass([FullPeepholeOptimise(), CommuteThroughMultis(), RemoveRedundancies(), rebase])
    comp.apply(tket_circuit)

    # Gates per Trotter step
    num_single_qubit_gates, num_two_qubit_gates = tket_circuit.n_1qb_gates(), tket_circuit.n_2qb_gates()
    print(f"1q gates: {num_single_qubit_gates}, 2q gates: {num_two_qubit_gates}")

    # Estimate number of Trotter steps required
    r_min, r_max = 1, 10
    while std_bin_trotter_error_sampling(H_spatial_search_padded, pauli_op, T, r_max, trotter_method, num_samples, num_jobs) > error_tol:
        r_max *= 2

    # binary search for r
    while r_max - r_min > 1:
        r = (r_min + r_max) // 2
        if std_bin_trotter_error_sampling(H_spatial_search_padded, pauli_op, T, r, trotter_method, num_samples, num_jobs) > error_tol:
            r_min = r
        else:
            r_max = r
    
    print(f"Finished N={N}, trotter steps={r_max}", flush=True)
    return num_single_qubit_gates, num_two_qubit_gates, r_max

def get_H_spatial_search(n, lamb, gamma, encoding, dimension):
    assert dimension == 2
    if encoding == "unary" or encoding == "antiferromagnetic":
        if encoding == "antiferromagnetic":
            assert n % 2 == 1, "only works for odd number of qubits"
        J = np.zeros((dimension * n, dimension * n))
        h = np.zeros(dimension * n)

        for i in range(dimension):
            h[i * n] = 1
            if encoding == "unary":
                h[(i + 1) * n - 1] = -1
            else:
                h[(i + 1) * n - 1] = (-1) ** (n)
            
            for j in np.arange(i * n, (i + 1) * n - 1):
                if encoding == "unary":
                    J[j, j + 1] = -1
                else:
                    J[j, j + 1] = 1

        H_pen = lamb * (sum_J_zz(dimension * n, J) + sum_h_z(dimension * n, h))

        # Create the oracle Hamiltonian
        J = np.zeros((dimension * n, dimension * n))
        J[n-1, n] = 1/4

        h = np.zeros(dimension * n)
        h[n-1] = 1/4
        h[n] = -1/4

        H_oracle = sum_h_z(dimension * n, h) + sum_J_zz(dimension * n, J)

        # Correction term for Laplacian
        h_correction = np.zeros(dimension * n)
        for i in range(dimension):
            h_correction[i * n] = 1/2
            if encoding == "unary":
                h_correction[(i + 1) * n - 1] = -1/2
            else:
                h_correction[(i + 1) * n - 1] = -(-1) ** (n+1) / 2
        H_correction = (sum_h_z(dimension * n, h_correction))
        H_adj = sum_x(dimension * n)
        H_laplacian = H_adj + H_correction

        return -gamma * H_laplacian + H_oracle + H_pen
    elif encoding == "one-hot":

        # Create the oracle Hamiltonian
        J = np.zeros((dimension * n, dimension * n))
        J[n-1, n] = -1/4
        h = np.zeros(dimension * n)
        h[n-1] = 1/4
        h[n] = 1/4
        H_oracle = sum_J_zz(dimension * n, J) + sum_h_z(dimension * n, h)

        # Laplacian correction term
        h = np.zeros(dimension * n)
        for i in range(dimension):
            h[i * n] = 1/2
            h[(i+1) * n - 1] = 1/2

        # Adjacency matrix
        J = np.zeros((dimension * n, dimension * n))
        for i in range(dimension):
            for j in np.arange(i * n, (i + 1) * n - 1):
                J[j,j+1] = 1
        
        H_laplacian = (sum_J_xx(dimension * n, J) + sum_J_yy(dimension * n, J)) / 2 - sum_h_z(dimension * n, h)

        return -gamma * H_laplacian + H_oracle
    else:
        raise ValueError("Encoding not supported")
    
def get_H_pauli_op(n, lamb, gamma, encoding, dimension):
    H = []

    if encoding == "unary" or encoding == "antiferromagnetic":
        if encoding == "antiferromagnetic":
            assert n % 2 == 1, "only works for odd number of qubits"

        for i in range(dimension):
            op = dimension * n * ['I']
            op[i * n] = 'Z'
            H.append(SparsePauliOp(''.join(op), lamb))
            if encoding == "unary":
                op = dimension * n * ['I']
                op[(i + 1) * n - 1] = 'Z'
                H.append(SparsePauliOp(''.join(op), -lamb))
            else:
                op = dimension * n * ['I']
                op[(i + 1) * n - 1] = 'Z'
                H.append(SparsePauliOp(''.join(op), ((-1) ** n) * lamb))
            
            for j in np.arange(i * n, (i + 1) * n - 1):
                if encoding == "unary":
                    op = dimension * n * ['I']
                    op[j] = 'Z'
                    op[j + 1] = 'Z'
                    H.append(SparsePauliOp(''.join(op), - lamb))
                else:                    
                    op = dimension * n * ['I']
                    op[j] = 'Z'
                    op[j + 1] = 'Z'
                    H.append(SparsePauliOp(''.join(op), lamb))

        # Create the oracle Hamiltonian
        op = dimension * n * ['I']
        op[n-1] = 'Z'
        op[n] = 'Z'
        H.append(SparsePauliOp(''.join(op), 1/4))

        op = dimension * n * ['I']
        op[n-1] = 'Z'
        H.append(SparsePauliOp(''.join(op), 1/4))
        op = dimension * n * ['I']
        op[n] = 'Z'
        H.append(SparsePauliOp(''.join(op), -1/4))

        # Correction term for Laplacian
        for i in range(dimension):
            op = dimension * n * ['I']
            op[i * n] = 'Z'
            H.append(SparsePauliOp(''.join(op), - gamma / 2))
            if encoding == "unary":
                op = dimension * n * ['I']
                op[(i + 1) * n - 1] = 'Z'
                H.append(SparsePauliOp(''.join(op), gamma / 2))
            else:
                op = dimension * n * ['I']
                op[(i + 1) * n - 1] = 'Z'
                H.append(SparsePauliOp(''.join(op), gamma * ((-1) ** (n+1)) / 2))
        
        for i in range(dimension * n):
            op = dimension * n * ['I']
            op[i] = 'X'
            H.append(SparsePauliOp(''.join(op), - gamma))

        return sum(H).simplify()
    elif encoding == "one-hot":

        # Create the oracle Hamiltonian
        op = dimension * n * ['I']
        op[n-1] = 'Z'
        op[n] = 'Z'
        H.append(SparsePauliOp(''.join(op), - 1/4))

        h = np.zeros(dimension * n)
        op = dimension * n * ['I']
        op[n-1] = 'Z'
        H.append(SparsePauliOp(''.join(op), 1/4))
        op = dimension * n * ['I']
        op[n] = 'Z'
        H.append(SparsePauliOp(''.join(op), 1/4))

        # Laplacian correction term
        for i in range(dimension):
            op = dimension * n * ['I']
            op[i * n] = 'Z'
            H.append(SparsePauliOp(''.join(op), gamma / 2))

            op = dimension * n * ['I']
            op[(i+1) * n - 1] = 'Z'
            H.append(SparsePauliOp(''.join(op), gamma / 2))

        # Adjacency matrix
        for i in range(dimension):
            for j in np.arange(i * n, (i + 1) * n - 1):
                op = dimension * n * ['I']
                op[j] = 'X'
                op[j+1] = "X"
                H.append(SparsePauliOp(''.join(op), - gamma / 2))

                op = dimension * n * ['I']
                op[j] = 'Y'
                op[j+1] = "Y"
                H.append(SparsePauliOp(''.join(op), - gamma / 2))

        return sum(H).simplify()
    else:
        raise ValueError("Encoding not supported")

def get_spatial_search_circuit(N, lamb, gamma, t, r, encoding, dimension, trotter_method):
    n = num_qubits_per_dim(N, encoding)

    dt = t / r

    circuit = Circuit()
    if encoding == "unary":
    
        if trotter_method == "first_order":
            for _ in range(r):
                # x rotations
                for i in range(dimension * n):
                    circuit.rx(i, - 2 * gamma * dt)

                # penalty term
                for i in range(dimension):
                    circuit.rz(i * n, 2 * lamb * dt)
                    circuit.rz((i + 1) * n - 1, - 2 * lamb * dt)
                    
                    for j in np.arange(i * n, (i + 1) * n - 1):
                        circuit.zz(j, j+1, -2 * lamb * dt)

                # oracle term
                circuit.zz(n-1, n, dt / 2)
                circuit.rz(n-1, dt / 2)
                circuit.rz(n, -dt / 2)

                # laplacian correction term
                for i in range(dimension):
                    circuit.rz(i * n, -gamma * dt)
                    circuit.rz((i + 1) * n - 1, gamma * dt)
            
        elif trotter_method == "second_order":
            for _ in range(r):
                # x rotations
                for i in range(dimension * n):
                    circuit.rx(i, - gamma * dt)

                # penalty term
                for i in range(dimension):
                    circuit.rz(i * n, 2 * lamb * dt)
                    circuit.rz((i + 1) * n - 1, - 2 * lamb * dt)
                    
                    for j in np.arange(i * n, (i + 1) * n - 1):
                        circuit.zz(j, j+1, -2 * lamb * dt)

                # oracle term
                circuit.zz(n-1, n, dt / 2)
                circuit.rz(n-1, dt / 2)
                circuit.rz(n, -dt / 2)

                # laplacian correction term
                for i in range(dimension):
                    circuit.rz(i * n, -gamma * dt)
                    circuit.rz((i + 1) * n - 1, gamma * dt)
                    
                # x rotations
                for i in range(dimension * n):
                    circuit.rx(i, - gamma * dt)
        
        elif trotter_method == "randomized_first_order":
            np.random.seed(t * r)
            for _ in range(r):
                if np.random.rand() < 0.5:
                    # x rotations
                    for i in range(dimension * n):
                        circuit.rx(i, - 2 * gamma * dt)

                    # penalty term
                    for i in range(dimension):
                        circuit.rz(i * n, 2 * lamb * dt)
                        circuit.rz((i + 1) * n - 1, - 2 * lamb * dt)
                        
                        for j in np.arange(i * n, (i + 1) * n - 1):
                            circuit.zz(j, j+1, -2 * lamb * dt)

                    # oracle term
                    circuit.zz(n-1, n, dt / 2)
                    circuit.rz(n-1, dt / 2)
                    circuit.rz(n, -dt / 2)

                    # laplacian correction term
                    for i in range(dimension):
                        circuit.rz(i * n, -gamma * dt)
                        circuit.rz((i + 1) * n - 1, gamma * dt)
                else:
                    
                    # penalty term
                    for i in range(dimension):
                        circuit.rz(i * n, 2 * lamb * dt)
                        circuit.rz((i + 1) * n - 1, - 2 * lamb * dt)
                        
                        for j in np.arange(i * n, (i + 1) * n - 1):
                            circuit.zz(j, j+1, -2 * lamb * dt)

                    # oracle term
                    circuit.zz(n-1, n, dt / 2)
                    circuit.rz(n-1, dt / 2)
                    circuit.rz(n, -dt / 2)

                    # laplacian correction term
                    for i in range(dimension):
                        circuit.rz(i * n, -gamma * dt)
                        circuit.rz((i + 1) * n - 1, gamma * dt)

                    # x rotations
                    for i in range(dimension * n):
                        circuit.rx(i, - 2 * gamma * dt)
        else:
            raise ValueError("Trotter order must be 1 or 2")
    elif encoding == "one-hot":
        marked_vertex_index_1 = N-1
        marked_vertex_index_2 = 0
        if trotter_method == "first_order":
            for _ in range(r):
                # Laplacian term
                for i in range(dimension):
                    for j in np.arange(i * n, (i + 1) * n - 1, 2):
                        circuit.xx(j, j + 1, - gamma * dt)
                        circuit.yy(j, j + 1, - gamma * dt)
                    for j in np.arange(i * n + 1, (i + 1) * n - 1, 2):
                        circuit.xx(j, j + 1, - gamma * dt)
                        circuit.yy(j, j + 1, - gamma * dt)
                        
                # Laplacian correction term
                for i in range(dimension):
                    circuit.rz(i * n, gamma * dt)
                    circuit.rz((i + 1) * n - 1, gamma * dt)

                # Oracle term
                circuit.zz(marked_vertex_index_1, n + marked_vertex_index_2, - dt / 2)
                circuit.rz(marked_vertex_index_1, dt / 2)
                circuit.rz(n + marked_vertex_index_2, dt / 2)

        elif trotter_method == "second_order":
            for _ in range(r):
                # Laplacian term
                for i in range(dimension):
                    for j in np.arange(i * n, (i + 1) * n - 1, 2):
                        circuit.xx(j, j + 1, - 0.5 * gamma * dt)
                        circuit.yy(j, j + 1, - 0.5 * gamma * dt)
                    for j in np.arange(i * n + 1, (i + 1) * n - 1, 2):
                        circuit.xx(j, j + 1, - 0.5 * gamma * dt)
                        circuit.yy(j, j + 1, - 0.5 * gamma * dt)
                        
                # Laplacian correction term
                for i in range(dimension):
                    circuit.rz(i * n, gamma * dt)
                    circuit.rz((i + 1) * n - 1, gamma * dt)

                # Oracle term
                circuit.zz(marked_vertex_index_1, n + marked_vertex_index_2, - dt / 2)
                circuit.rz(marked_vertex_index_1, dt / 2)
                circuit.rz(n + marked_vertex_index_2, dt / 2)

                # Laplacian term
                for i in range(dimension):
                    for j in np.arange(i * n + 1, (i + 1) * n - 1, 2):
                        circuit.xx(j, j + 1, - 0.5 * gamma * dt)
                        circuit.yy(j, j + 1, - 0.5 * gamma * dt)
                    for j in np.arange(i * n, (i + 1) * n - 1, 2):
                        circuit.xx(j, j + 1, - 0.5 * gamma * dt)
                        circuit.yy(j, j + 1, - 0.5 * gamma * dt)
                        
        elif trotter_method == "randomized_first_order":
            
            np.random.seed(t * r)
            for _ in range(r):
                if np.random.rand() < 0.5:
                    # Laplacian term
                    for i in range(dimension):
                        for j in np.arange(i * n, (i + 1) * n - 1, 2):
                            circuit.xx(j, j + 1, - gamma * dt)
                            circuit.yy(j, j + 1, - gamma * dt)
                        for j in np.arange(i * n + 1, (i + 1) * n - 1, 2):
                            circuit.xx(j, j + 1, - gamma * dt)
                            circuit.yy(j, j + 1, - gamma * dt)
                            
                    # Laplacian correction term
                    for i in range(dimension):
                        circuit.rz(i * n, gamma * dt)
                        circuit.rz((i + 1) * n - 1, gamma * dt)

                    # Oracle term
                    circuit.zz(marked_vertex_index_1, n + marked_vertex_index_2, - dt / 2)
                    circuit.rz(marked_vertex_index_1, dt / 2)
                    circuit.rz(n + marked_vertex_index_2, dt / 2)

                else:
                    # Oracle term
                    circuit.zz(marked_vertex_index_1, n + marked_vertex_index_2, - dt / 2)
                    circuit.rz(marked_vertex_index_1, dt / 2)
                    circuit.rz(n + marked_vertex_index_2, dt / 2)

                    # Laplacian correction term
                    for i in range(dimension):
                        circuit.rz(i * n, gamma * dt)
                        circuit.rz((i + 1) * n - 1, gamma * dt)

                    # Laplacian term
                    for i in range(dimension):
                        for j in np.arange(i * n + 1, (i + 1) * n - 1, 2):
                            circuit.xx(j, j + 1, - gamma * dt)
                            circuit.yy(j, j + 1, - gamma * dt)
                        for j in np.arange(i * n, (i + 1) * n - 1, 2):
                            circuit.xx(j, j + 1, - gamma * dt)
                            circuit.yy(j, j + 1, - gamma * dt)
    else:
        raise ValueError("Encoding not supported")
    return circuit

In [10]:
print("Resource estimation for 2D spatial search.")
num_jobs = 64
print("Number of jobs:", num_jobs)
num_samples = 5000

dimension = 2
trotter_method = "second_order"

print(f"Method: {trotter_method}")

Resource estimation for 2D spatial search.
Number of jobs: 64
Method: second_order


In [11]:
N = 4
r = 12
# Unary encoding
encoding = "unary"
device = LocalSimulator()

print(f"Running N = {N}", flush=True)
n = num_qubits_per_dim(N, encoding)
codewords = get_codewords(N, dimension, encoding, periodic=False)

# Compute the optimal gamma
L = get_laplacian_lattice(N, d=dimension)
marked_vertex_1 = np.zeros(N)
marked_vertex_1[N-1] = 1
marked_vertex_2 = np.zeros(N)
marked_vertex_2[0] = 1

marked_vertex = np.kron(marked_vertex_1, marked_vertex_2)
H_oracle = - csc_matrix(np.outer(marked_vertex, marked_vertex))
# Sign is flipped here; this function minimizes the difference between the two largest eigenvalues of gamma * L + H_oracle
gamma = scipy_get_optimal_gamma(L, -H_oracle, 0.3)
H_spatial_search = - gamma * L + H_oracle
T = get_T_2d(N, H_spatial_search)
print(f"T={T}")

lamb = 2

H_ebd = get_H_spatial_search(n, lamb, gamma, encoding, dimension)
error_tol = estimate_trotter_error(N, H_ebd, T, get_spatial_search_circuit(N, lamb, gamma, T, r, encoding, dimension, trotter_method), dimension, encoding, codewords, device, num_samples, num_jobs)

# Standard binary
print("\nRunning resource estimation for standard binary encoding", flush=True)
binary_one_qubit_gate_count_per_trotter_step, binary_two_qubit_gate_count_per_trotter_step, binary_trotter_steps = get_binary_resource_estimate(N, dimension, error_tol, trotter_method, num_samples, num_jobs)
print("Std binary gate counts")
print(binary_one_qubit_gate_count_per_trotter_step * binary_trotter_steps, binary_two_qubit_gate_count_per_trotter_step * binary_trotter_steps)


Running N = 4
T=4.894117647058824

Running resource estimation for standard binary encoding
N = 4
1q gates: 277, 2q gates: 41
Finished N=4, trotter steps=3
Std binary gate counts
831 123


In [13]:
N = 5
r = 5
# One-hot encoding
encoding = "one-hot"
device = LocalSimulator()


print(f"Running N = {N}", flush=True)
n = num_qubits_per_dim(N, encoding)
codewords = get_codewords(N, dimension, encoding, periodic=False)

# Compute the optimal gamma
L = get_laplacian_lattice(N, d=dimension)
marked_vertex_1 = np.zeros(N)
marked_vertex_1[N-1] = 1
marked_vertex_2 = np.zeros(N)
marked_vertex_2[0] = 1

marked_vertex = np.kron(marked_vertex_1, marked_vertex_2)
H_oracle = - csc_matrix(np.outer(marked_vertex, marked_vertex))
# Sign is flipped here; this function minimizes the difference between the two largest eigenvalues of gamma * L + H_oracle
gamma = scipy_get_optimal_gamma(L, -H_oracle, 0.3)
H_spatial_search = - gamma * L + H_oracle
T = get_T_2d(N, H_spatial_search)
print(f"T={T}")

lamb = None

H_ebd = get_H_spatial_search(n, lamb, gamma, encoding, dimension)

error_tol = estimate_trotter_error(N, H_ebd, T, get_spatial_search_circuit(N, lamb, gamma, T, r, encoding, dimension, trotter_method), dimension, encoding, codewords, device, num_samples, num_jobs)

# Standard binary
print("\nRunning resource estimation for standard binary encoding", flush=True)
binary_one_qubit_gate_count_per_trotter_step, binary_two_qubit_gate_count_per_trotter_step, binary_trotter_steps = get_binary_resource_estimate(N, dimension, error_tol, trotter_method, num_samples, num_jobs)

print("Std binary gate counts")
print(binary_one_qubit_gate_count_per_trotter_step * binary_trotter_steps, binary_two_qubit_gate_count_per_trotter_step * binary_trotter_steps)


Running N = 5
T=6.568627450980392

Running resource estimation for standard binary encoding
N = 5
1q gates: 4350, 2q gates: 744
Finished N=5, trotter steps=6
Std binary gate counts
26100 4464


# Real space

In [14]:
def get_real_space_H(N, a, b):
    p_sq = np.zeros((N,N))
    x_sq = np.zeros((N,N))
    x = np.zeros((N,N))

    # \hat{p}^2
    for j in range(N-1):
        p_sq[j,j] = 0.5 * (2 * j + 1)
    for j in range(N-2):
        p_sq[j,j+2] = -0.5 * np.sqrt((j+1) * (j+2))
        p_sq[j+2,j] = -0.5 * np.sqrt((j+1) * (j+2))
    
    # \hat{x}^2
    for j in range(N-1):
        x_sq[j,j] = 0.5 * (2 * j + 1)
    for j in range(N-2):
        x_sq[j,j+2] = 0.5 * np.sqrt((j+1) * (j+2))
        x_sq[j+2,j] = 0.5 * np.sqrt((j+1) * (j+2))
    
    # \hat{x}
    for j in range(N-1):
        x[j,j+1] = np.sqrt((j+1)/np.sqrt(2))
        x[j+1,j] = np.sqrt((j+1)/np.sqrt(2))

    H = 0.5 * p_sq + 0.5 * a * x_sq + b * x
    return H

def get_unary_real_space_H_ebd(N, a, b):
    n = num_qubits_per_dim(N, encoding="unary")

    J = np.zeros((n,n))
    for i in range(n-1):
        J[i, i+1] = np.sqrt((i + 1) * (i + 2))
    p_sq = sum_delta_n(n, np.ones(n)) - 0.5 * sum_J_xx(n, J)
    x_sq = sum_delta_n(n, np.ones(n)) + 0.5 * sum_J_xx(n, J)

    h = np.array([np.sqrt((j+1) / 2) for j in range(n)])
    x = sum_h_x(n, h)

    H_ebd = 0.5 * p_sq + 0.5 * a * x_sq + b * x
    return H_ebd

def get_H_pauli_op(N, a, b, encoding):

    n = num_qubits_per_dim(N, encoding=encoding)

    H = []
    if encoding == "unary":
        J = np.zeros((n,n))
        for i in range(n-1):
            op = n * ['I']
            op[i] = 'X'
            op[i+1] = 'X'
            H.append(SparsePauliOp(''.join(op), 0.25 * (a - 1) * np.sqrt((i+1) * (i+2))))

            J[i, i+1] = np.sqrt((i + 1) * (i + 2))
        
        for i in range(n):
            op = n * ['I']
            op[i] = 'Z'
            H.append(SparsePauliOp(''.join(op), -0.25 * (1 + a)))

        for j in range(n):
            op = n * ['I']
            op[j] = 'X'
            H.append(SparsePauliOp(''.join(op), b * np.sqrt((j+1) / 2)))

    elif encoding == "one-hot":
        for j in range(n-1):
            op = n * ['I']
            op[j] = 'X'
            op[j+1] = 'X'
            H.append(SparsePauliOp(''.join(op), b * np.sqrt((j+1) / 2)))

        for j in range(n-2):
            op = n * ['I']
            op[j] = 'X'
            op[j+2] = 'X'
            H.append(SparsePauliOp(''.join(op), 0.25 * (a - 1) * np.sqrt((j+1) * (j+2))))
        
        for j in range(n):
            op = n * ['I']
            op[j] = 'Z'
            H.append(SparsePauliOp(''.join(op), -2 * (j+1/2)))

    return sum(H).simplify()

def get_binary_resource_estimate(N, T, error_tol, a, b, trotter_method, num_samples, num_jobs):
    
    print(f"N = {N}", flush=True)

    H_padded = get_real_space_H(2 ** int(np.ceil(np.log2(N))), a, b)
    for i in np.arange(N, 2 ** int(np.ceil(np.log2(N)))):
        for j in range(N):
            H_padded[i,j] = 0
            H_padded[j,i] = 0
    # H = get_real_space_H(N, a, b)
    # H_padded = np.pad(H, (0, 2 ** int(np.ceil(np.log2(N))) - N))

    pauli_op = SparsePauliOp.from_operator(H_padded)

    # Compute number of gates per Trotter step
    if trotter_method == "first_order" or trotter_method == "randomized_first_order":
        circuit = LieTrotter(reps=1).synthesize(PauliEvolutionGate(pauli_op.group_commuting()))
    elif trotter_method == "second_order":
        circuit = SuzukiTrotter(order=2, reps=1).synthesize(PauliEvolutionGate(pauli_op.group_commuting()))
    else:
        raise ValueError(f"{trotter_method} not supported")
    
    compiled_circuit = transpile(circuit, basis_gates=['rxx', 'rx', 'ry', 'rz'], optimization_level=3)
    tket_circuit = qiskit_to_tk(compiled_circuit)
    gateset = {OpType.Rx, OpType.Ry, OpType.Rz, OpType.XXPhase}
    rebase = auto_rebase_pass(gateset) 
    comp = SequencePass([FullPeepholeOptimise(), CommuteThroughMultis(), RemoveRedundancies(), rebase])
    comp.apply(tket_circuit)

    # Gates per Trotter step
    num_single_qubit_gates, num_two_qubit_gates = tket_circuit.n_1qb_gates(), tket_circuit.n_2qb_gates()
    print(f"1q gates: {num_single_qubit_gates}, 2q gates: {num_two_qubit_gates}")

    # Estimate number of Trotter steps required
    r_min, r_max = 1, 10
    while std_bin_trotter_error_sampling(H_padded, pauli_op, T, r_max, trotter_method, num_samples, num_jobs) > error_tol:
        r_max *= 2

    # binary search for r
    while r_max - r_min > 1:
        r = (r_min + r_max) // 2
        if std_bin_trotter_error_sampling(H_padded, pauli_op, T, r, trotter_method, num_samples, num_jobs) > error_tol:
            r_min = r
        else:
            r_max = r
    
    print(f"Finished N={N}, trotter steps={r_max}", flush=True)
    return num_single_qubit_gates, num_two_qubit_gates, r_max


In [15]:
print("Resource estimation for real-space simulation.")

num_jobs = 64
print("Number of jobs:", num_jobs)
num_samples = 5000

dimension = 1
trotter_method = "randomized_first_order"
print(f"Error tolerance: {error_tol : 0.2f}.")
print(f"Method: {trotter_method}")

a, b = 2, -1/2
T = 5
N = 5

Resource estimation for real-space simulation.
Number of jobs: 64
Error tolerance:  1.01.
Method: randomized_first_order


In [16]:
encoding = "one-hot"
r = 11
print(f"\nRunning resource estimation for {encoding} encoding", flush=True)
device = LocalSimulator()

print(f"N = {N} for {encoding}", flush=True)

H = get_real_space_H(N, a, b)
n = num_qubits_per_dim(N, encoding)
codewords = get_codewords(N, dimension, encoding, periodic=False)

H_terms, graph = get_H_terms_one_hot(N, H)
H_ebd = sum(H_terms)
    
error_tol = estimate_trotter_error(N, H_ebd, T, get_one_hot_circuit(N, H, T, r, trotter_method), dimension, encoding, codewords, device, num_samples, num_jobs)

print("\nRunning resource estimation for standard binary encoding")
binary_one_qubit_gate_count_per_trotter_step, binary_two_qubit_gate_count_per_trotter_step, binary_trotter_steps = get_binary_resource_estimate(N, T, error_tol, a, b, trotter_method, num_samples, num_jobs)

print("Std binary gate counts")
print(binary_one_qubit_gate_count_per_trotter_step * binary_trotter_steps, binary_two_qubit_gate_count_per_trotter_step * binary_trotter_steps)



Running resource estimation for one-hot encoding
N = 5 for one-hot

Running resource estimation for standard binary encoding
N = 5
1q gates: 166, 2q gates: 20
Finished N=5, trotter steps=11
Std binary gate counts
1826 220
